# **Lecture 4: Data Cleaning**
---

### **Description**
In this notebook, we will explore how to find and handle null values.

<br>

### **Structure**
**Part 1**: [Finding Nulls](#p1)

**Part 2**: [Removing Nulls](#p2)

**Part 3**: [Imputing Nulls](#p3)



<br>

### **Learning Objectives**
By the end of this notebook, we will:
* Be able to code to find nulls using a variety of Pandas EDA techniques.
* Be able to code to remove and replace null values.



<br>

**Run the code below before continuing.**


In [5]:
!pip install pandas
!pip install numpy
!pip install numpy

import pandas as pd
import numpy as np
import random 

# Set seed for reproducibility
random.seed(123)
np.random.seed(123)

# Function to introduce random NaNs
def introduce_nan(series, nan_fraction=0.1):
    n_nan = int(len(series) * nan_fraction)
    nan_indices = random.sample(range(len(series)), n_nan)
    series.iloc[nan_indices] = np.nan
    return series

# Create a larger dataset with some messy data
n_rows = 100
data = {
    'student_id': np.arange(1, n_rows + 1),  # Unique IDs
    'name': [random.choice(['Angelique', 'Brianna', 'Cristina', 'Daisy', 'Eve', 'Frank']) for _ in range(n_rows)],  # Random names
    'age': [random.choice([12, 14, 15, 16, 17, 18, np.nan]) for _ in range(n_rows)],  # Random ages with NaNs and empty strings
    'grade': [random.choice(['A', 'B', 'C', 'D', 'F', 'Z', '', 'incomplete', np.nan]) for _ in range(n_rows)],  # Grades with NaNs and 'incomplete'
    'absences': [random.choice([0, 1, 2, 3, np.nan]) for _ in range(n_rows)],  # Absences with NaNs and empty strings
    'detention_hours': [random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 100,  np.nan, 'none']) for _ in range(n_rows)],  # Detention hours with NaNs and 'none'
}

# Create DataFrame
messy_df = pd.DataFrame(data)

# Introduce random NaNs in some of the columns
messy_df['name'] = introduce_nan(messy_df['name'], nan_fraction=0.1)
messy_df['grade'] = introduce_nan(messy_df['grade'], nan_fraction=0.2)
messy_df['absences'] = introduce_nan(messy_df['absences'], nan_fraction=0.1)
messy_df['detention_hours'] = introduce_nan(messy_df['detention_hours'], nan_fraction=0.15)

# Output the messy dataset
messy_df.head(10) # Display first 10 rows for review

/tmp/ipykernel_80/4035337818.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series.iloc[nan_indices] = np.nan
/tmp/ipykernel_80/4035337818.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series.iloc[nan_indices] = np.nan
/tmp/ipykernel_80/4035337818.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series.iloc[nan_indices] = np.nan
/tmp/ipykernel_80/4035337818.py:17: SettingWithCopyWarning: 
A value is trying to be set on

,student_id,name,age,grade,absences,detention_hours
0,1,Angelique,17.0,C,NaN,NaN
1,2,Cristina,17.0,NaN,3.0,1
2,3,Angelique,18.0,,2.0,6
3,4,Daisy,NaN,A,0.0,0
4,5,Cristina,12.0,C,3.0,NaN
5,6,Angelique,18.0,NaN,NaN,6
6,7,Angelique,15.0,A,2.0,10
7,8,NaN,16.0,NaN,1.0,5
8,9,Eve,NaN,incomplete,NaN,8
9,10,Eve,18.0,incomplete,2.0,0


<a name="p1"></a>

---
##**Part 1: Finding Missing Values**
---

#### **Problem #1.1**

Using `.head()`, what do you initially notice about the data?

In [6]:
messy_df.head()

,student_id,name,age,grade,absences,detention_hours
0,1,Angelique,17.0,C,NaN,NaN
1,2,Cristina,17.0,NaN,3.0,1
2,3,Angelique,18.0,,2.0,6
3,4,Daisy,NaN,A,0.0,0
4,5,Cristina,12.0,C,3.0,NaN


#### **Problem #1.2**

Using `.info()`, explore how many nulls each column has.

What do you notice?

In [8]:
# COMPLETE THIS CODE
messy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   student_id       100 non-null    int64  
 1   name             90 non-null     object 
 2   age              88 non-null     float64
 3   grade            68 non-null     object 
 4   absences         74 non-null     float64
 5   detention_hours  80 non-null     object 
dtypes: float64(2), int64(1), object(3)
memory usage: 4.8+ KB


#### **Problem #1.3**
Which column(s) do you think we could filter using conditionals?

#### **Problem #1.3**
**Together**, let's filter some columns by what normal values we think it should have.

**NOTE**: Most of our columns are not numeric, but we want to apply numeric logic to them. Use `pd.to_numeric()` to convert your columns for the filter.
* Example: `(pd.to_numeric(messy_df['age'], errors='coerce') < 13)`

In [9]:
# COMPLETE THIS CODE
pd.to_numeric(messy_df['age'], errors ='coerce')

0     17.0
1     17.0
2     18.0
3      NaN
4     12.0
      ... 
95    14.0
96    17.0
97    15.0
98    17.0
99    12.0
Name: age, Length: 100, dtype: float64

#### **Problem #1.3**
**Together**, let's filter the data using `isna()`.

In [10]:
# COMPLETE THIS CODE
outliers_ages = messy_df[(pd.to_numeric(messy_df['age'], errors = 'coerce') < 13)]

<a name="p2"></a>

---
##**Part 2: Dropping Nulls**
---

#### **Problem #2.1**

**Together**, let's drop all nulls from the dataframe row-wise using `dropna()`.

**Note**: `dropna()` does not change our original dataframe, but it instead makes a copy. We either have to save the new dataframe in another variable or set the parameter `inplace = True` which changes our original dataframe. We are going to create a new variable because we want to use the original dataset later on and don't want to permanently modify it.

In [11]:
# COMPLETE THIS CODE
messy_df.dropna()

,student_id,name,age,grade,absences,detention_hours
2,3,Angelique,18.0,,2.0,6
6,7,Angelique,15.0,A,2.0,10
9,10,Eve,18.0,incomplete,2.0,0
10,11,Cristina,18.0,F,3.0,0
13,14,Brianna,16.0,C,2.0,none
15,16,Cristina,14.0,Z,3.0,100
20,21,Brianna,16.0,Z,0.0,4
22,23,Daisy,12.0,B,0.0,10
24,25,Eve,16.0,Z,3.0,5
27,28,Angelique,15.0,B,3.0,0


In [15]:
nonulls_df = messy_df.dropna()
nonulls_df.shape

(32, 6)

#### **Problem #2.2**
What do you notice about our dataset now that we've dropped all rows that contained nulls?

<a name="p3"></a>

---
##**Part 3: Imputing Nulls**
---

#### **Problem #3.1**
**Together**, let's impute the missing `age` values with the mean.

In [16]:
# COMPLETE THIS CODE
mean_age = messy_df['age'].mean()
messy_df['age'] = messy_df['age'].fillna(mean_age)
mean_age

np.float64(15.329545454545455)

#### **Problem #3.2**
**Together**, let's fill the missing `absences` values with the median.

In [17]:
median_absences = messy_df['absences'].median()
messy_df['absences'] = messy_df['absences'].fillna(median_absences)
messy_df['absences']

0     2.0
1     3.0
2     2.0
3     0.0
4     3.0
     ... 
95    1.0
96    1.0
97    1.0
98    2.0
99    1.0
Name: absences, Length: 100, dtype: float64

#### **Problem #3.3**
Not all imputations need to be the mean or median, and it can depend on the data what we use to fill the missing values. **Together**, let's impute the missing `detention_hours` values with 0 because we might assume a student has no detention hours if they're missing.

In [20]:
messy_df['detention_hours'] = messy_df['detention_hours'].fillna
messy_df['detention_hours']

0     <bound method NDFrame.fillna of 0     <bound m...
1     <bound method NDFrame.fillna of 0     <bound m...
2     <bound method NDFrame.fillna of 0     <bound m...
3     <bound method NDFrame.fillna of 0     <bound m...
4     <bound method NDFrame.fillna of 0     <bound m...
                            ...                        
95    <bound method NDFrame.fillna of 0     <bound m...
96    <bound method NDFrame.fillna of 0     <bound m...
97    <bound method NDFrame.fillna of 0     <bound m...
98    <bound method NDFrame.fillna of 0     <bound m...
99    <bound method NDFrame.fillna of 0     <bound m...
Name: detention_hours, Length: 100, dtype: object

#### **Problem #3.4**
**Together**, let's fill the missing `grade` values with the value `Incomplete`.

In [22]:
messy_df.fillna({'grade':'Incomplete'}, inplace = True)
messy_df['grade']

0              C
1     Incomplete
2               
3              A
4              C
         ...    
95    Incomplete
96             Z
97             A
98              
99             A
Name: grade, Length: 100, dtype: object

In [23]:
#End of notebook
---
© 2024 The Coding School, All rights reserved

SyntaxError: invalid character '©' (U+00A9) (2083094837.py, line 3)